In [15]:
%config Completer.use_jedi = False
import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

world_geojson = json.load(open('countries.geojson', 'r'))
covid_df = pd.read_csv('full_grouped.csv')

covid_df['confirmed_log'] = np.log10(covid_df['confirmed'] + 1e-10)

geojson_name_changes = {
    "United States of America": "US",
    "Democratic Republic of the Congo": "Congo (Kinshasa)" ,
    "Republic of Congo": "Congo (Brazzaville)",
    "United Republic of Tanzania":"Tanzania",
}
for feature in world_geojson['features']:
    old_name = feature['properties']['ADMIN']
    new_name = geojson_name_changes.get(old_name, old_name)
    feature['properties']['ADMIN'] = new_name
csv_name_changes = {
    "Burma" : "Myanmar",
    "Cote d'Ivoire" : "Ivory Coast"
}
covid_df['country'] = covid_df['country'].replace(csv_name_changes)

#picking out the country names from the geojson
geojson_country_names = [feature['properties']['ADMIN'] for feature in world_geojson['features']]
# print(geojson_country_names)
# converting the csv and geojson country names in sets to compare
geojson_country_set = set(geojson_country_names)
csv_country_set = set(covid_df['country'])
#print(csv_country_set)
# making a set of common countries
common_countries = geojson_country_set.intersection(csv_country_set)

#filtering out the features list from the world_geojson 
filtered_geojson_features =  [feature for feature in world_geojson['features'] if feature['properties']['ADMIN'] in common_countries]

#making a new geojson to use
filtered_geojson = {'type' : 'FeatureCollection', 'features' : filtered_geojson_features}
# temp = set([feature['properties']['ADMIN'] for feature in filtered_geojson['features']])
# print(temp)

#making a new csv of countries
filtered_csv_countries = covid_df[covid_df['country'].isin(common_countries)]
#verifyling the equality of the 2 sets of data
# print( set(temp) - set(filtered_csv_countries['country'].unique()))

#assigning unique ids to each geojson country
country_id_map = {}
for feature in filtered_geojson['features']:
    feature['id'] = feature['properties']['ISO_A3']
    country_id_map[feature['properties']['ADMIN']] = feature['id']
#adding that id to csv, while supressing settingWithCopyWarning
with pd.option_context('mode.chained_assignment', None):
    filtered_csv_countries.loc[:, 'id'] = filtered_csv_countries['country'].apply(lambda x:country_id_map[x])

# country = 'Andorra'
# country_data = filtered_csv_countries[filtered_csv_countries['country'] == country]
# country_data
date_to_check = '2020-07-23'
date_data = filtered_csv_countries[filtered_csv_countries['date'] == date_to_check]
# filtered_geojson['features'][0].keys()

fig = px.choropleth_mapbox(date_data,
                    locations="id",
                    geojson=filtered_geojson, 
                    color='confirmed_log',
                    hover_name='country',
                    hover_data=['confirmed', 'deaths'],
                    mapbox_style='carto-positron',
                    zoom = 1.5,
                    opacity = 0.7)
fig.show()

In [16]:
import pandas as pd
import numpy as np
import plotly
import plotly.io as pio
pio.renderers.default = 'browser'


# Reading sample data using pandas DataFrame
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv")

data = [dict(type='choropleth',
			locations = df['code'].astype(str),
			z=df['total exports'].astype(float),
			locationmode='USA-states')]

# let's create some more additional data
for i in range(5):
	data.append(data[0].copy())
	data[-1]['z'] = data[0]['z'] * np.random.rand(*data[0]['z'].shape)

# let's now create slider for map
steps = []
for i in range(len(data)):
	step = dict(method='restyle',
				args=['visible', [False] * len(data)],
				label='Year {}'.format(i + 1980))
	step['args'][1][i] = True
	steps.append(step)

slider = [dict(active=0,
				pad={"t": 1},
				steps=steps)]	
layout = dict(geo=dict(scope='usa',
					projection={'type': 'albers usa'}),
			sliders=slider)

fig = dict(data=data,
		layout=layout)

# Render the Plotly figure in a new browser tab
# fig.show()
plotly.offline.iplot(fig)

Opening in existing browser session.
